Para poder efectuar el EDA, vamos a unir los 7 archivos trabajados en un solo archivo.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
dfbalancefinal = pd.read_csv('df_balance.csv')

In [3]:
dfbalancefinal.head(3)

,cliente_id,t_id,balance_prev,balance_post
0,C1231006815,0,170136.0,160296.36
1,M1979787155,0,0.0,0.00
2,C1666544295,1,21249.0,19384.72


In [4]:
dfbalancefinal.shape

(12725240, 4)

In [5]:
dfbalancefinal.duplicated(subset=['cliente_id', 't_id']).value_counts()

False    12725240
dtype: int64

In [6]:
df_cuantia = pd.read_csv('df_cuantia.csv')

In [7]:
df_cuantia.head(3)

,t_id,cuantia
0,0,9839.64
1,1,1864.28
2,2,181.00


In [8]:
df_cuantia.shape

(6362620, 2)

In [9]:
df_cuantia.duplicated(subset=['t_id']).value_counts()

False    6362620
dtype: int64

In [10]:

df_clientes = pd.read_csv('df_clientes.csv')

In [11]:
df_clientes.head(3)

,t_id,origen,destino
0,0,C1231006815,M1979787155
1,1,C1666544295,M2044282225
2,2,C1305486145,C553264065


In [12]:
df_clientes.shape

(6362620, 3)

In [13]:
df_clientes.duplicated(subset=['t_id']).value_counts()

False    6362620
dtype: int64

Unión dataset balance y dataset clientes

In [14]:
clientes_origen = df_clientes[['t_id', 'origen']]

In [15]:
clientes_origen = clientes_origen.rename(columns={'origen':'cliente_id'
                                   })

In [16]:
clientes_origen.head(3)

,t_id,cliente_id
0,0,C1231006815
1,1,C1666544295
2,2,C1305486145


In [17]:
df_balance_clientes_origen =  pd.merge(dfbalancefinal ,clientes_origen, how = 'inner', on = ('t_id', 'cliente_id'))

In [18]:
df_balance_clientes_origen_1 = df_balance_clientes_origen.rename(columns={'cliente_id':'id_cliente_origen','balance_prev':'balance_prev_origen',
                                                                          'balance_post':'balance_post_origen'}) 

In [19]:
df_balance_clientes_origen_1.shape

(6362620, 4)

In [20]:
df_balance_clientes_origen_1.head(3)

,id_cliente_origen,t_id,balance_prev_origen,balance_post_origen
0,C1231006815,0,170136.0,160296.36
1,C1666544295,1,21249.0,19384.72
2,C1305486145,2,181.0,0.00


In [21]:
df_balance_clientes_origen_1.duplicated(subset=['t_id']).value_counts()

False    6362620
dtype: int64

In [22]:
clientes_destino = df_clientes[['t_id', 'destino']]

In [23]:
clientes_destino = clientes_destino.rename(columns={'destino':'cliente_id'
                                   })

In [24]:
df_balance_clientes_destino =  pd.merge(dfbalancefinal ,clientes_destino, how = 'inner', on = ('t_id', 'cliente_id'))

In [25]:
df_balance_clientes_destino_1 = df_balance_clientes_destino.rename(columns={'cliente_id':'id_cliente_destino','balance_prev':'balance_prev_destino',
                                                                            'balance_post':'balance_post_destino'})

In [26]:
df_balance_clientes_destino_1.duplicated(subset=['t_id']).value_counts()

False    6362620
dtype: int64

In [27]:
df_balance_clientes =  pd.merge(df_balance_clientes_origen_1,df_balance_clientes_destino_1, how = 'inner', on = ('t_id'))

In [28]:
df_balance_clientes.shape

(6362620, 7)

In [29]:
df_balance_clientes.duplicated(subset=['t_id']).value_counts()

False    6362620
dtype: int64

In [30]:

df_balance_clientes.head(3)

,id_cliente_origen,t_id,balance_prev_origen,balance_post_origen,id_cliente_destino,balance_prev_destino,balance_post_destino
0,C1231006815,0,170136.0,160296.36,M1979787155,0.0,0.0
1,C1666544295,1,21249.0,19384.72,M2044282225,0.0,0.0
2,C1305486145,2,181.0,0.00,C553264065,0.0,0.0


In [31]:
df_balance_clientes_1 = df_balance_clientes[['t_id','id_cliente_origen','balance_prev_origen','balance_post_origen',
                                             'id_cliente_destino','balance_prev_destino','balance_post_destino']]

Unimos el dataset que ahora tiene el balance de origen y destino con su transacción, con la cuantía de la transacción

In [32]:
df_balance_clientes_cuantia =  pd.merge(df_balance_clientes_1 ,df_cuantia, how = 'inner', on = ('t_id'))

In [33]:
df_balance_clientes_cuantia.shape

(6362620, 8)

Cargamos el dataset tipo y unimos con el dataset que tiene los datos del balance y de la cuantía

In [35]:
df_tipo = pd.read_csv('df_tipo.csv')

In [36]:
df_balance_tipo =  pd.merge(df_balance_clientes_cuantia ,df_tipo, how = 'inner', on = ('t_id'))

In [37]:
df_balance_tipo.shape

(6362620, 9)

Cargamos el dataset tiempo y unimos con el dataset que tiene los datos del balance, de la cuantía de la trancción y su tipo

In [38]:
df_tiempo = pd.read_csv('df_tiempo.csv')

In [39]:
df_balance_tiempo =  pd.merge(df_balance_tipo ,df_tiempo, how = 'inner', on = ('t_id'))

Cargamos el dataset alarma_fraude y unimos con el dataset que tiene los datos del balance, de la cuantía de la trancción, su tipo y tiempo.

In [40]:
df_alarma = pd.read_csv('df_alarma_fraude.csv')

In [41]:
df_alarma.shape

(16, 2)

In [42]:
df_balance_alarma = pd.merge(df_balance_tiempo ,df_alarma, how = 'left', on = ('t_id'))

In [43]:
df_balance_alarma.shape

(6362620, 12)

Chequeamos una transacción donde saltó la alarma de fraude

In [44]:
a = df_balance_alarma.loc[df_balance_alarma['t_id'] == 2736446]

In [45]:
a

,t_id,id_cliente_origen,balance_prev_origen,balance_post_origen,id_cliente_destino,balance_prev_destino,balance_post_destino,cuantia,tipo,index,fecha_final,mensaje_alarma
2741452,2736446,C728984460,4953893.08,4953893.08,C639921569,0.0,0.0,4953893.08,TRANSFER,2736446,2022-09-10 03:00:00,Detectado_fraude


In [47]:
df_balance_alarma = df_balance_alarma.drop(columns = ['index'])

Cargamos el dataset es_fraude y unimos con el dataset que tiene los datos del balance, de la cuantía de la trancción, su tipo, tiempo y si se detecto la alarma de fraude.

In [48]:
df_fraude = pd.read_csv('df_es_fraude.csv')

In [49]:

df_fraude.shape

(6362520, 2)

In [50]:

df_balance_fraude = pd.merge(df_balance_alarma ,df_fraude, how = 'left', on = ('t_id'))

In [51]:
df_balance_fraude.shape

(6362620, 12)

In [52]:
df_transacciones = df_balance_fraude[['t_id','tipo','fecha_final','id_cliente_origen','balance_prev_origen','balance_post_origen','id_cliente_destino','balance_prev_destino',
                                      'balance_post_destino','cuantia', 'mensaje_alarma','es_fraude']]

In [53]:
df_transacciones.sample(5)

,t_id,tipo,fecha_final,id_cliente_origen,balance_prev_origen,balance_post_origen,id_cliente_destino,balance_prev_destino,balance_post_destino,cuantia,mensaje_alarma,es_fraude
6095013,6094206,PAYMENT,2022-09-23 01:00:00,C1303897228,260.00,0.00,M1017413617,0.00,0.00,7648.81,NaN,False
5827665,5826126,TRANSFER,2022-09-18 01:00:00,C1718586375,56797.14,0.00,C735598592,6344845.99,7180780.70,835934.71,NaN,False
3515126,3510124,PAYMENT,2022-09-12 02:00:00,C5631925,198545.00,192898.88,M70045007,0.00,0.00,5646.12,NaN,False
1489396,1485636,CASH_OUT,2022-09-07 04:00:00,C9415316,1686.00,0.00,C1498574559,2466.74,40894.64,38427.90,NaN,False
4165499,4160892,CASH_OUT,2022-09-13 23:00:00,C1758439504,427281.00,292041.57,C362953375,0.00,135239.43,135239.43,NaN,False


In [54]:
df_transacciones.duplicated(subset=[ 't_id']).value_counts()

False    6362620
dtype: int64

In [56]:
df_transacciones.isnull().sum()

t_id                          0
tipo                          0
fecha_final                   0
id_cliente_origen             0
balance_prev_origen           0
balance_post_origen           0
id_cliente_destino            0
balance_prev_destino          0
balance_post_destino          0
cuantia                       0
mensaje_alarma          6362604
es_fraude                   100
dtype: int64

Filtramos los 100 casos de fraude que no estan etiquetados y debemos predecir con el algoritmo de ML 

In [57]:
fraudes_a_predecir = df_transacciones[df_transacciones.es_fraude.isnull()]

In [58]:
fraudes_a_predecir.shape

(100, 12)

In [77]:
fraudes_a_predecir.to_csv('fraudes_a_predecir.csv', index = False)

Para efectuar el EDA trabajamos con el dataset sin los casos a predecir 

In [69]:
df = df_transacciones[df_transacciones.es_fraude.notnull()]

In [70]:
df.shape

(6362520, 12)

Reemplazamos el mensaje de la columna mensaje_alarma por True 

In [71]:
df.mensaje_alarma.unique()

array([nan, 'Detectado_fraude'], dtype=object)

In [72]:
df = df.replace({"Detectado_fraude": True})

En los casos que no salto la alarma de fraude vamos a rellenar los nulos con False 

In [74]:
df.fillna({'mensaje_alarma': False}, inplace=True)

In [75]:
df.head(3)

,t_id,tipo,fecha_final,id_cliente_origen,balance_prev_origen,balance_post_origen,id_cliente_destino,balance_prev_destino,balance_post_destino,cuantia,mensaje_alarma,es_fraude
0,0,PAYMENT,2022-09-01 08:00:00,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,9839.64,False,False
1,1,PAYMENT,2022-09-01 08:00:00,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,1864.28,False,False
2,2,TRANSFER,2022-09-01 08:00:00,C1305486145,181.0,0.00,C553264065,0.0,0.0,181.00,False,True


In [76]:
df.isnull().sum()

t_id                    0
tipo                    0
fecha_final             0
id_cliente_origen       0
balance_prev_origen     0
balance_post_origen     0
id_cliente_destino      0
balance_prev_destino    0
balance_post_destino    0
cuantia                 0
mensaje_alarma          0
es_fraude               0
dtype: int64

In [78]:
df.to_csv('df_completo.csv', index = False)